In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# 팔꿈치 데이터 불러오기
back_elbow = 'coordinates\\1_back_elbow_coordinates.csv'
bdata = pd.read_csv(back_elbow)
right_elbow = 'coordinates\\1_right_elbow_coordinates.csv'
rdata = pd.read_csv(right_elbow)
print(bdata.head())
print(rdata.head())

          x         y
0  0.559086  0.531539
1  0.555475  0.530428
2  0.555496  0.529788
3  0.555362  0.528827
4  0.555379  0.528082
          x         y
0  0.171110  0.477804
1  0.173505  0.477811
2  0.175027  0.477754
3  0.175894  0.477759
4  0.176424  0.477825


In [4]:
# 둘의 y좌표가 안 맞는다...아마 카메라간 각도의 차이 및 구도의 차이 때문에 생기는 오차 같다
# 일단 y좌표중 하나를 기준으로 잡고 설정
# bdata의 x좌표를 x, y좌표를 z로 바꾸고, rdata의 x를 z로 바꾸고 세 개의 데이터를 합친다

# brdata 새로운 데이터프레임에 생성
brdata = pd.DataFrame()
brdata[['x', 'z']] = bdata[['x', 'y']]
brdata[['y']] = rdata[['x']]


# 컬럼 순서 변경
brdata = brdata[['x', 'y', 'z']]

brdata.head()

,x,y,z
0,0.559086,0.171110,0.531539
1,0.555475,0.173505,0.530428
2,0.555496,0.175027,0.529788
3,0.555362,0.175894,0.528827
4,0.555379,0.176424,0.528082


In [9]:
import cv2

path = 'sample_videos\\1_back.mp4'
cap = cv2.VideoCapture(path)

# 영상 사이즈 재조정
width = 480
height = 800

frame_number = 0
paused = False

while True:
    if not paused:
        ret, frame = cap.read()
        frame_number += 1

    if ret:  
        frame = cv2.resize(frame, (width, height))

        # 프레임 수 화면에 표시
        cv2.putText(frame, f"Frame: {frame_number}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('video', frame)
        
    else:
        print("Failed to retrieve frame")
        break

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('p'):  # p 누르면 영상 일시정지 기능 추가
        paused = not paused  

cap.release()
cv2.destroyAllWindows()

In [12]:
# 1번 영상의 경우 시작 : 190 프레임 종료 : 220프레임으로 설정
# 190번째 행과 220번째 행의 좌표 추출
# 190번째 행의 좌표
elstart = brdata.iloc[190]
# 220번째 행의 좌표
elend = brdata.iloc[220]
print(elstart)
print(elend)

x    0.476520
y    0.197982
z    0.525808
Name: 190, dtype: float64
x    0.475219
y    0.198831
z    0.490253
Name: 220, dtype: float64


In [13]:
# 어깨 데이터 불러오기
back_shoulder = 'coordinates\\1_back_shoulder_coordinates.csv'
bdata1 = pd.read_csv(back_shoulder)
right_shoulder = 'coordinates\\1_right_shoulder_coordinates.csv'
rdata1 = pd.read_csv(right_shoulder)
print(bdata1.head())
print(rdata1.head())

          x         y
0  0.458757  0.453358
1  0.456473  0.450861
2  0.458258  0.450469
3  0.458726  0.450281
4  0.458728  0.450338
          x         y
0  0.212694  0.436688
1  0.212631  0.436080
2  0.212598  0.435687
3  0.212590  0.435473
4  0.212684  0.435288


In [14]:
# 어깨도 좌표 맞춰보기
br1data = pd.DataFrame()
br1data[['x', 'z']] = bdata1[['x', 'y']]
br1data[['y']] = rdata1[['x']]
br1data = br1data[['x', 'y', 'z']]
br1data.head()

,x,y,z
0,0.458757,0.212694,0.453358
1,0.456473,0.212631,0.450861
2,0.458258,0.212598,0.450469
3,0.458726,0.212590,0.450281
4,0.458728,0.212684,0.450338


In [15]:
brdata.head()

,x,y,z
0,0.559086,0.171110,0.531539
1,0.555475,0.173505,0.530428
2,0.555496,0.175027,0.529788
3,0.555362,0.175894,0.528827
4,0.555379,0.176424,0.528082


In [16]:
# brdata와 br1data 의 190번째 행과 220번째 행의 좌표값을 벡터로 추출
# 190번째 행의 좌표
shstart = br1data.iloc[190]
# 220번째 행의 좌표
shend = br1data.iloc[220]
print(shstart)
print(shend)

x    0.502348
y    0.230367
z    0.491745
Name: 190, dtype: float64
x    0.366219
y    0.234273
z    0.465363
Name: 220, dtype: float64


In [20]:
# elstart-shstart 벡터값 추출 및 절대값 도출
elshstart = elstart - shstart
print(elshstart)
elshend = elend - shend
print(elshend)

x   -0.025827
y   -0.032385
z    0.034063
Name: 190, dtype: float64
x    0.109000
y   -0.035441
z    0.024890
Name: 220, dtype: float64


In [23]:
# 두 벡터값 내적 구하기
# 주어진 두 벡터
v1 = pd.Series({'x': -0.025827, 'y': -0.032385, 'z': 0.034063})
v2 = pd.Series({'x': 0.109000, 'y': -0.035441, 'z': 0.024890})

# 내적
dp = (v1 * v2).sum()

# 벡터의 크기
mag_v1 = np.sqrt((v1**2).sum())
mag_v2 = np.sqrt((v2**2).sum())

# 벡터의 크기를 이용한 코사인값 계산
cos_angle = dp / (mag_v1 * mag_v2)

# 코사인값을 이용한 각도 계산
angle_rad = np.arccos(cos_angle)

# 라디안 각도를 도로 변환
angle_deg = np.degrees(angle_rad)
print(angle_deg)

97.48653693023982
